In [1]:
import os
import json
import pandas as pd
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,com.datastax.spark:spark-cassandra-connector_2.12:3.1.0 pyspark-shell'

In [2]:
from ast import literal_eval

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SSKafka") \
    .getOrCreate()
    
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
dsraw = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test") \
  .option("startingOffsets", "latest") \
  .load()

In [4]:
### Insert into Cassandra
spark.conf.set("spark.cassandra.connection.host","127.0.0.1:9042")
# spark.conf.set("spark.sql.extensions","com.datastax.spark.connector.CassandraSparkExtensions")
spark.conf.set("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog")

In [5]:
ds = dsraw.selectExpr("CAST(value AS STRING)")

In [6]:
print(type(dsraw))
print(type(ds))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [7]:
alertQuery = ds \
        .writeStream \
        .queryName("qalerts")\
        .format("memory")\
        .start()

In [16]:
alerts = spark.sql("select * from qalerts")
# alerts.tail(15)

In [17]:
alerts.count()

10

In [18]:
df_data = alerts.toPandas()

In [19]:
df5 = df_data.copy()

In [26]:
df_cass = pd.DataFrame()
for i, row in df5.iterrows():
    s = row.value.replace("'", "\"")
    s1 = json.loads(s)
#     print(s1)
    if ('user_id' in s1):
        print ("processing user :", s1['user_id'])
        s1['user_created'] = s1['user_created'][:23]
#         s1['user_comms'] = S1['user_comms']
        if (len(df_cass) == 0):
            df_cass = pd.DataFrame.from_records([s1])
#             pd.Series(df_cass).to_frame()
        else:
            df_cass = df_cass.append(s1, ignore_index=True)

processing user : U1612554
processing user : U1037314
processing user : U1936209
processing user : U1137354
processing user : U1967163
processing user : U1089595
processing user : U1596384
processing user : U1013979
processing user : U1074338
processing user : U1076031


In [27]:
df_cass.columns

Index(['user_id', 'user_nm', 'user_city', 'user_st', 'user_interests',
       'user_comms', 'user_comm_cnt', 'user_created'],
      dtype='object')

In [28]:
df_cass.drop_duplicates(subset=['user_id', 'user_nm', 'user_city', 'user_st', 'user_interests',
       'user_comm_cnt', 'user_created'], inplace=True)
df_cass.reset_index(drop=True, inplace=True)

In [30]:
df_cass

,user_id,user_nm,user_city,user_st,user_interests,user_comms,user_comm_cnt,user_created
0,U1612554,Rati Bali,Nellore,Chhattisgarh,swimming,sports,6,2021-08-01 19:06:41.614
1,U1037314,Emir Dara,Bongaigaon,Gujarat,chess,sports,6,2021-08-01 19:06:41.618
2,U1936209,Miraya Karpe,Pallavaram,Jharkhand,technology,science,7,2021-08-01 19:06:41.605
3,U1137354,Parinaaz Tak,Tumkur,Jharkhand,kickboxing,science,7,2021-08-01 19:06:41.614
4,U1967163,Ehsaan Hari,Bhopal,Karnataka,swimming,tech,4,2021-08-01 19:06:41.625
5,U1089595,Tara Handa,Bhilwara,Telangana,badminton,science,7,2021-08-01 19:06:41.628
6,U1596384,Kaira Dhar,Sagar,Maharashtra,cricket,sports,6,2021-08-01 19:06:41.632
7,U1013979,Kismat Saini,Asansol,Mizoram,mobiles,science,7,2021-08-01 19:06:41.621
8,U1074338,Neysa Ratti,Loni,Tamil Nadu,volleyball,tech,4,2021-08-01 19:06:41.635
9,U1076031,Mishti Shah,Ongole,Meghalaya,technology,tech,4,2021-08-01 19:06:41.641


In [31]:
sparkDF_cass=spark.createDataFrame(df_cass)

In [32]:
sparkDF_cass.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="users", keyspace="raw_stream")\
    .save()

In [93]:
# spark.read.format("org.apache.spark.sql.cassandra").options(table="users", keyspace="raw_stream").load().show()
# spark.sql.format

+-------+---+---------+-------------+----------+------------+--------------+-------+-------+
|user_id| id|user_city|user_comm_cnt|user_comms|user_created|user_interests|user_nm|user_st|
+-------+---+---------+-------------+----------+------------+--------------+-------+-------+
+-------+---+---------+-------------+----------+------------+--------------+-------+-------+



In [105]:
# spark.sql("CREATE DATABASE IF NOT EXISTS mycatalog.testks WITH DBPROPERTIES (class='SimpleStrategy',replication_factor='1')")
# spark.sql("CREATE TABLE mycatalog.testks.testtab (key Int, value STRING) USING cassandra PARTITIONED BY (key)")

# # //List their contents
# spark.sql("SHOW NAMESPACES FROM mycatalog").show
# spark.sql("SHOW TABLES FROM mycatalog.testks").show
